# Tapis v3 Gateways 2021 Hands-on

In this notebook, you will use Tapis v3 to create two systems and one application that will be used to run
an image classification job on both a VM and an HPC cluster.

To execute each `In[#]` cell, you can click inside the cell and press `Shift + Enter`

## Enter training account information

To get things started, please run the following and enter the training account information provided to you:

In [ ]:
import getpass

tenant = 'tacc'
base_url = 'https://' + tenant + '.tapis.io'

username = input('Username: ')
password = getpass.getpass(prompt='Password: ', stream=None)
host = input('Host: ')

## Authenticate and initialize Tapis v3 client

Using this information, you can now use `tapipy` to authenticate in the tenant and initialize the
Tapis v3 client. You should see your token information displayed.

In [ ]:
from tapipy.tapis import Tapis
#Create python Tapis client for user
client = Tapis(base_url= base_url, username=username, password=password)
# *** Tapis v3: Call to Tokens API
client.get_tokens()
# Print Tapis v3 token
client.access_token

## Systems

In this section we create two Tapis systems, one for a training VM host and one for the stampede2 HPC cluster.

Note that although it is possible, we have not provided any login credentials in the system definitions.
Well-crafted system definitions are likely to be copied and re-used, so, for security reasons, it is recommended that
login credentials be registered using separate API calls as discussed below.

### Create a system for the VM host

In [ ]:
user_id = username
system_id = "gateways21-vm-" + user_id
vm_password = password

# Create the system definition
exec_system_vm = {
  "id": system_id,
  "description": "System for testing jobs on a VM for gateways2021 tutorial",
  "systemType": "LINUX",
  "host": "129.114.19.192",
  "defaultAuthnMethod": "PASSWORD",
  "rootDir": "/home/"+user_id,
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "workdir",
  "batchScheduler": "SLURM",
  "jobIsBatch": True,
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "debug",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ],
  "batchDefaultLogicalQueue": "tapisNormal"
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id)
print("****************************************************")
client.systems.createSystem(**exec_system_vm)

In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id)
print("****************************************************")
client.systems.getSystem(systemId=system_id)

### Register Credentials for the VM system

After creating the system, you will need to register credentials for your username. These will be used by Tapis to
access the host. Various authentication methods can be used to access a system, such as PASSWORD and PKI_KEYS. Here we
will cover registering a password.

In [ ]:
# Register credentials
client.systems.createUserCredential(systemId=system_id, userName=user_id, password=vm_password)

Now you can use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
# List files at the rootDir for the system
client.files.listFiles(systemId=system_id, path="/")

### Create a system for the HPC cluster

With just a few changes to the system definition you can create a second system that can be used to run the
same application on an HPC cluster. Note the minimal changes:

* **id** - A unique id is required
* **host** - This is now the main hostname for the HPC cluster
* **defaultAuthnMethod** - Accessing the HPC cluster requires an SSH key pair
* **rootDir** - Using the root directory of the host gives us flexibility in setting **jobWorkingDir**. Note that you still need LINUX permissions.
* **jobWorkingDir** - Now determined dynamically using the Tapis v3 function HOST_EVAL()
* **batchLogicalQueue.hpcQueueName** - A different underlying queue name is used for the HPC cluster.

In [ ]:
user_id = username
system_id = "gateways21-hpc-" + user_id

# Create the system definition
exec_system_hpc = {
  "id": system_id,
  "description": "System for testing jobs on an HPC cluster for gateways2021 tutorial",
  "systemType": "LINUX",
  "host": "stampede2.tacc.utexas.edu",
  "defaultAuthnMethod": "PKI_KEYS",
  "rootDir": "/",
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "HOST_EVAL($WORK2)",
  "batchScheduler": "SLURM",
  "jobIsBatch": True,
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "normal",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ],
  "batchDefaultLogicalQueue": "tapisNormal"
}

# Use the client to create the system in Tapis
print("****************************************************")
print("Create system: " + system_id)
print("****************************************************")
client.systems.createSystem(**exec_system_hpc)

In [ ]:
# List all systems available to you
print("****************************************************")
print("List all systems")
print("****************************************************")
client.systems.getSystems()

In [ ]:
# Get details for the system you created
print("****************************************************")
print("Fetch system: " + system_id)
print("****************************************************")
client.systems.getSystem(systemId=system_id)

### Register Credentials for the HPC system

As before, now you will need to register credentials for your username. These will be used by Tapis to
access the host. For this HPC cluster the authentication method is PKI_KEYS so you will need to register a PKI
keypair.

Then use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
hpc_privateKey = "***"
hpc_publicKey = "***"
# Register credentials
client.systems.createUserCredential(systemId=system_id, userName=user_id, publicKey=hpc_publicKey, privateKey=hpc_privateKey)
# List files at the rootDir for the system
client.files.listFiles(systemId=system_id, path="/")

## Applications

In order to run a job on a system you will need to create a Tapis application.